# The Movie DataBase (TMDB)

In [32]:
import pandas as pd
import numpy as np
import json

In [12]:
movies= pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [10]:
credits= pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [20]:
movies['genres'].head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [41]:
a=movies['genres'].apply(json.loads).values.tolist()
pd.DataFrame.from_records(a)

,0,1,2,3,4,5,6
0,"{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 878, 'name': 'Science Fiction'}",None,None,None
1,"{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 28, 'name': 'Action'}",None,None,None,None
2,"{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}","{'id': 80, 'name': 'Crime'}",None,None,None,None
3,"{'id': 28, 'name': 'Action'}","{'id': 80, 'name': 'Crime'}","{'id': 18, 'name': 'Drama'}","{'id': 53, 'name': 'Thriller'}",None,None,None
4,"{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}","{'id': 878, 'name': 'Science Fiction'}",None,None,None,None
5,"{'id': 14, 'name': 'Fantasy'}","{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}",None,None,None,None
6,"{'id': 16, 'name': 'Animation'}","{'id': 10751, 'name': 'Family'}",None,None,None,None,None
7,"{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}","{'id': 878, 'name': 'Science Fiction'}",None,None,None,None
8,"{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 10751, 'name': 'Family'}",None,None,None,None
9,"{'id': 28, 'name': 'Action'}","{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}",None,None,None,None


In [54]:
a=pd.read_json(movies['genres'][1])
b=movies['id'][1]
a['movie_id']=b
a

,id,name,movie_id
0,12,Adventure,285
1,14,Fantasy,285
2,28,Action,285


In [ ]:
def split_list(occupation):
    return occupation.split(', ')

In [48]:

movies['genres'][1]

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

1- For each movie, compute the number of cast members

2- How many movies do not have a homepage?

3- For each year, how many movies do not have a homepage?

4- Extract the domain of each homepage.

5- Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

6- For each movie, compute the gross margin (difference between revenue and budget)

7- For each movie, compute the number of crew members

8- For each movie, compute the number of directors

9- For each language, compute the number of movies where such language is spoken.

10- For each company and each decade, compute the overall revenue

11- For each decade, compute the company with maximum revenue

12- In each year, how many movies have revenue smaller than the budget?



1- Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

2- For each actor find the total revenue attributed to him/her.

3- Find the actor that is responsible for the most overall revenue.



1- For each movie, compute the ratio between males and females in the cast

2- For each movie, compute the ratio between the attributed revenue of males and females in the cast

3- Find the director that has the highest average ratio computed in the previous point.
